## Pydantic

In [3]:
import os
from langchain.chat_models import init_chat_model
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
model= init_chat_model("groq:qwen/qwen3-32b")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002AD9E635D00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002AD9E635FD0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from pydantic import BaseModel,Field

class Movie(BaseModel):
    title:str=Field(description="The title of the movie")
    year:int=Field(description="This year of the movie was released")
    director:str=Field(description="Director of the movie")
    rating:float=Field(description="The movie rating out of 10")

In [5]:
model_with_structure= model.with_structured_output(Movie)
model_with_structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002AD9E635D00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002AD9E635FD0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'This year of the movie was released', 'type': 'integer'}, 'director': {'description': 'Director of the movie', 'type': 'string'}, 'rating': {'description': 'The movie rating out of 10', 'type': 'number'}}, 'required': ['title', 'year', 'di

In [6]:
model.invoke("Provide details about Movie Inception")

AIMessage(content='<think>\nOkay, I need to provide details about the movie Inception. Let me start by recalling what I know. Inception is a 2010 film directed by Christopher Nolan. It’s a sci-fi action movie. The main character is Dom Cobb, played by Leonardo DiCaprio. He’s a thief who enters people’s dreams to steal secrets. The term for that is a thief, I think. The plot involves him being offered a chance to erase his criminal past by performing the reverse: planting an idea into someone\'s mind, which is called "inception." \n\nThe supporting cast includes Joseph Gordon-Levitt, who plays Arthur, Cobb’s right-hand man. There\'s also Tom Hardy as Eames, who can impersonate people. Ellen Page as Ariadne, who’s the architect and helps design the dream. And Tom Berenger as a character named... wait, I think his character is a government agent or something. Maybe a CIA guy. Also, there\'s Dileep Rao as a tech guy, maybe the dream-sharing tech operator.\n\nThe plot revolves around Cobb t

In [7]:
model_with_structure.invoke("Provide details about Movie Inception")

Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8)

### Message input with parse output

In [9]:
from pydantic import BaseModel,Field

class Movie(BaseModel):
    title:str=Field(description="The title of the movie")
    year:int=Field(description="This year of the movie was released")
    director:str=Field(description="Director of the movie")
    rating:float=Field(description="The movie rating out of 10")

model_with_structure= model.with_structured_output(Movie, include_raw=True)

model_with_structure.invoke("Provide details about Movie Inception")

{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for details about the movie Inception. Let me check what functions I have available. There\'s a Movie function that requires title, year, director, and rating. I need to make sure I have all that information for Inception.\n\nFirst, the title is obviously "Inception". The year it was released was 2010. The director is Christopher Nolan. As for the rating, I think it\'s around 8.8 on IMDb. Let me confirm that. Yes, IMDb gives it an 8.8/10. So I have all the required parameters. I should structure the tool call with these details. Make sure the JSON is correctly formatted with the right data types: title and director as strings, year as an integer, and rating as a number. No need for any extra information since the user just wants the details provided by the function. Alright, that should cover it.\n', 'tool_calls': [{'id': '9f142848s', 'function': {'arguments': '{"director":"Christopher Nolan

### Nested Structure 

In [12]:
from pydantic import BaseModel

class Actor(BaseModel):
    name:str
    role:str

class MovieDetails(BaseModel):
    title:str
    year:int
    cast:list[Actor]
    genres:list[str]
    budget:float | None = Field(None, description="Budget in millions USD")


model_with_structure= model.with_structured_output(MovieDetails)

response=model_with_structure.invoke("Provide details about Movie Inception")
response



MovieDetails(title='Inception', year=2010, cast=[Actor(name='Leonardo DiCaprio', role='Dom Cobb'), Actor(name='Joseph Gordon-Levitt', role='Arthur'), Actor(name='Elliot Page', role='Ariadne'), Actor(name='Tom Hardy', role='Bane')], genres=['Action', 'Sci-Fi', 'Thriller'], budget=160.0)

### TypedDict (Not much focus on Runtimr validation)

In [19]:
from typing_extensions import TypedDict, Annotated

class MovieDict(TypedDict):
    title:Annotated[str,"The title of the movie"]
    year:Annotated[int,"This year of the movie was released"]
    director:Annotated[str,"Director of the movie"]
    rating:Annotated[float,"The movie rating out of 10"]

model_with_typedict= model.with_structured_output(MovieDict)

response=model_with_typedict.invoke("Provide details about Movie Inception")
response

{'director': 'Christopher Nolan',
 'rating': 8.8,
 'title': 'Inception',
 'year': 2010}

In [20]:
class Actor(TypedDict):
    name:str
    role:str

class MovieDetails(TypedDict):
    title:str
    year:int
    cast:list[Actor]
    genres:list[str]
    budget:float | None = Field(None, description="Budget in millions USD")


model_with_structure= model.with_structured_output(MovieDetails)

response=model_with_structure.invoke("Provide details about Movie Inception")
response


{'budget': 160000000,
 'cast': [{'name': 'Leonardo DiCaprio', 'role': 'Dom Cobb'},
  {'name': 'Joseph Gordon-Levitt', 'role': 'Arthur'},
  {'name': 'Ellen Page', 'role': 'Mal'}],
 'genres': ['Science Fiction', 'Action', 'Thriller'],
 'title': 'Inception',
 'year': 2010}

In [21]:
import os
from dotenv import load_dotenv
load_dotenv()


os.environ["GEMINI_API_KEY"]=os.getenv("GEMINI_API_KEY")

### Dataclasses

In [1]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI


class ContactInfo(BaseModel):
    """Contact information for  a Person"""
    name:str=Field(description="The name of the person")
    email:str=Field(description="The email address of the person")
    phone:str=Field(description="The phone number of the person")

model= ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)
agent=create_agent(
    model=model,
    response_format=ContactInfo
)

result=agent.invoke({
    "messages":[{"role" :"user" , "content" : "Extract contact info from John Doe, john@example.com , (555) 123-4678"}]
})

print(result["structured_response"])

name='John Doe' email='john@example.com' phone='(555) 123-4678'


In [2]:
from dataclasses import dataclass
from langchain.agents import create_agent

@dataclass
class ContactInfo:
    name:str
    email:str
    phone:str

agent=create_agent(
    model=model,
    response_format=ContactInfo
)
result=agent.invoke({
    "messages":[{"role" :"user" , "content" : "Extract contact info from John Doe, john@example.com , (555) 123-4678"}]
})

print(result["structured_response"])




ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4678')
